# Compute summary statistics on a Spark DataFrame using .summary() or dbutils data summaries


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# Crear SparkSession
spark = SparkSession.builder.appName("ResumenVinos").getOrCreate()

# Definir esquema
schema = StructType([
    StructField("nombre", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("alcohol", DoubleType(), True),
    StructField("acidez", DoubleType(), True),
    StructField("dulzor", DoubleType(), True)
])

# Datos de ejemplo
datos = [
    ("Frutal Rosado", "joven", 11.5, 3.2, 4.0),
    ("Dulce Mora", "joven", 12.0, 3.5, 6.0),
    ("Cítrico Blanco", "joven", 10.8, 3.8, 2.5),
    ("Suave Tropical", "joven", 11.2, 3.3, 5.0),
    ("Vino de Uva", "joven", 12.2, 3.6, 3.5)
]

# Crear DataFrame
df = spark.createDataFrame(data=datos, schema=schema)

In [0]:
# Aplicar describe
# df.describe().show()

# Aplicar summary (más completo)
df.summary().show()

In [0]:
dbutils.data.summarize(df)

In [0]:
display(df)

In [0]:
df

## 📌 Puntos clave
**.summary() (PySpark)**
✅ Se usa para obtener estadísticas resumidas de todas las columnas (numéricas y no numéricas).
✅ Proporciona: 'count', 'mean', 'stddev', 'min', '25%', '50%', '75%', 'max'
✅ Incluye percentiles → mejor que .describe().
✅ Requiere .show() para visualizar

**.describe() (comparación)(PySpark)**
Solo da: 'count', 'mean', 'stddev', 'min', 'max'
❌ No incluye percentiles
❌ Menos completa → no es suficiente para análisis exploratorio completo

**dbutils.data.summarize(df) (Databricks)**
✅ Abre una vista visual interactiva del DataFrame.
✅ Muestra automáticamente:
    Distribución de cada columna (gráficos)
    Estadísticas básicas y tipo de variable
    Recuentos para categorías
    Detección de outliers
✅ No necesita show(), pero solo funciona dentro de notebooks Databricks.

**display(df) (Databricks)**
Muestra la tabla con paginación.
Permite elegir visualizaciones:

✅ Histogram
✅ Bar chart
✅ Line chart
✅ Box plot
✅ Scatter plot
✅ Map (si hay coordenadas)
❌ No imprime estadísticas numéricas automáticamente

# Remove outliers from a Spark DataFrame based on standard deviation or IQR


##  Standard deviation

Un valor es outlier si está fuera del rango: **μ ± k ⋅ σ**

Donde:
- μ es la media
- σ es la desviación estándar
- k suele ser 2 o 3

Pasos en PySpark:
- Calcular mean y stddev con .agg()
- Filtrar con .filter() los valores fuera del rango

In [0]:
from pyspark.sql.functions import mean, stddev

mean_std = df.select(mean("price").alias("mean"), stddev("price").alias("stddev")).collect()[0]
mean_val, stddev_val = mean_std["mean"], mean_std["stddev"]
# Rango permitido
lower_bound = mean_val - 2 * stddev_val
upper_bound = mean_val + 2 * stddev_val
filtered_df = df.filter((df["price"] >= lower_bound) & (df["price"] <= upper_bound))

## IQR (Interquartile Range)

Usa los cuartiles Q1, Q3 y el IQR:

**IQR = Q3 − Q1**

Outliers:
- menores que Q1 − 1.5 ⋅ IQR
- mayores que Q3 + 1.5 ⋅ IQR

Pasos en PySpark:
- Obtener cuartiles con .approxQuantile()
- Calcular IQR
- Filtrar fuera de los límites

### Pyspark con .approxQuantile()

In [0]:
q1, q3 = df.approxQuantile("price", [0.25, 0.75], 0.0)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
filtered_df = df.filter((df["price"] >= lower_bound) & (df["price"] <= upper_bound))

### Pandas con .quantile()

In [0]:
# ejemplo en pandas 

Q1 = df['col'].quantile(0.25)
Q3 = df['col'].quantile(0.75)
IQR = Q3 - Q1
filtered_df = df[(df['col'] >= Q1 - 1.5 * IQR) & (df['col'] <= Q3 + 1.5 * IQR)]

## 📌 Puntos clave
¿Cuándo usar cada método?

| Método                  | Cuándo usar                                                                    |
| ----------------------- | ------------------------------------------------------------------------------ |
| **Desviación estándar** | Cuando la variable sigue una distribución **normal o simétrica**               |
| **IQR**                 | Cuando la variable tiene una distribución **sesgada o con outliers** evidentes |

**Nota:**

- **Distribución normal o simétrica:** Se representa gráficamente como una curva en forma de campana
- **Distribución sesgada con outliers:** Una distribución está sesgada cuando una de las colas de la curva es más larga que la otra

**Data Cleaning for Machine Learning** 
https://community.databricks.com/t5/technical-blog/data-cleaning-for-machine-learning/ba-p/95410#:~:text=Removing%20outliers%3A%20Using%20statistical%20methods,the%20overall%20impact%20of%20outliers


# Create visualizations for categorical or continuous features

## A. Con display(df) en notebooks
Método nativo de Databricks.

Muestra una tabla interactiva → puedes cambiar el tipo de gráfico desde el menú.
Soporta:
Bar chart
Histogram
Line chart
Box plot
Scatter plot
Maps (si hay datos de coordenad)

## B. Con pandas API on Spark (pyspark.pandas) o conversión a Pandas
Esto te permite usar .plot() como en pandas normal:

In [0]:
import pyspark.pandas as ps
psdf = df.pandas_api()
psdf["age"].plot(kind="hist")

####
pdf = df.toPandas()
pdf["price"].plot(kind="box")

## 📌 Puntos Clave

| Tipo de variable | Ejemplo                    | Tipo de gráfico más adecuado                    |
| ---------------- | -------------------------- | ----------------------------------------------- |
| **Categórica**   | género, país, clase social | Bar chart, Pie chart, Count plot                |
| **Continua**     | edad, precio, ingreso      | Histogram, Box plot, Density plot, Scatter plot |

**Tipo de gráfico según el objetivo**

** - Para variables categóricas:**

| Gráfico                            | Cuándo usar                              | Función                              |
| ---------------------------------- | ---------------------------------------- | ------------------------------------ |
| **Bar chart**                      | Comparar frecuencias de categorías       | `display()` → selecciona “Bar chart” |
| **Pie chart**                      | Comparar proporciones (pocas categorías) | ⚠️ Pocas veces útil                  |
| **Count plot** (en pandas/seaborn) | Contar ocurrencias                       | `sns.countplot()`                    |

** - Para variables continuas:**

| Gráfico          | Cuándo usar                                       | Función                                       |
| ---------------- | ------------------------------------------------- | --------------------------------------------- |
| **Histogram**    | Ver distribución                                  | `display()` o `psdf["col"].plot(kind="hist")` |
| **Box plot**     | Detectar outliers y asimetría                     | `display()` o `.plot(kind="box")`             |
| **Density plot** | Visualizar suavemente la forma de la distribución | `sns.kdeplot()` (pandas/seaborn)              |
| **Scatter plot** | Relación entre dos variables numéricas            | `display(df)` → “Scatter”                     |


https://docs.databricks.com/aws/en/visualizations




# Compare two categorical or two continuous features using the appropriate method

## 1. Comparar dos variables categóricas

Usa una tabla de contingencia y un test de chi-cuadrado si quieres evaluar independencia.


Ejemplo: Comparar genero y compra_realizada

Supongamos este DataFrame:

In [0]:
data = [
    ("F", "Sí"), ("M", "No"), ("F", "Sí"), ("M", "Sí"),
    ("F", "No"), ("M", "No"), ("F", "Sí"), ("M", "Sí")
]
df_spark = spark.createDataFrame(data, ["genero", "compra_realizada"])
df_spark.show()

a) Tabla de contingencia

In [0]:
pd.crosstab(df['genero'], df['compra_realizada'], margins=True)

b) Prueba de chi-cuadrado

In [0]:
# una forma 
from scipy.stats import chi2_contingency

tabla = pd.crosstab(df['genero'], df['compra_realizada'])
chi2, p, dof, expected = chi2_contingency(tabla)

print(f"Chi2: {chi2}, p-valor: {p}")

In [0]:
# con spark 
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.stat import ChiSquareTest

# Convertir texto a numérico
indexer1 = StringIndexer(inputCol="genero", outputCol="genero_idx")
indexer2 = StringIndexer(inputCol="compra_realizada", outputCol="compra_idx")

df_indexed = indexer1.fit(df_spark).transform(df_spark)
df_indexed = indexer2.fit(df_indexed).transform(df_indexed)

# Crear vector de características
assembler = VectorAssembler(inputCols=["genero_idx"], outputCol="features")
df_features = assembler.transform(df_indexed)

# Aplicar prueba chi-cuadrado
chi_result = ChiSquareTest.test(df_features, "features", "compra_idx")
chi_result.select("pValues", "degreesOfFreedom", "statistics").show(truncate=False)

Si el p-valor < 0.05, hay relación significativa entre las dos variables categóricas.

## 2. Comparar dos variables continuas

Usa la correlación (Pearson o Spearman).

Ejemplo: Comparar edad vs ingresos

In [0]:
data_continua = [
    (25, 2200), (32, 2700), (47, 3500), (51, 4000), (38, 3000)
]
df_continuo = spark.createDataFrame(data_continua, ["edad", "ingresos"])
df_continuo.show()

a) Correlación de Pearson

In [0]:
# Pearson (lineal)
df_continuo.stat.corr("edad", "ingresos", method="pearson")

b) Correlación de Spearman (si no es lineal)

In [0]:
# Spearman (monótona)
df_continuo.stat.corr("edad", "ingresos", method="spearman")

## 📌 Puntos clave

**Comparación final de métodos**

| Tipo de variables        | Método recomendado           | Visualización                       |
| ------------------------ | ---------------------------- | ----------------------------------- |
| Categórica vs categórica | `crosstab`, chi-cuadrado     | Heatmap de frecuencias, stacked bar |
| Continua vs continua     | Pearson/Spearman correlation | Scatter plot                        |

**Conceptos clave que debes dominar para el examen**
| Concepto                     | Detalle                                                               |
| ---------------------------- | --------------------------------------------------------------------- |
| `df.stat.crosstab()`         | Único método de Spark para conteo entre categorías                    |
| `chi2_contingency()` (scipy) | No está en Spark nativo, pero evaluado conceptualmente                |
| `df.stat.corr()`             | Aplica Pearson o Spearman en Spark                                    |
| Pearson vs Spearman          | Pearson: lineal + normalidad<br>Spearman: ordinal o relación monótona |

| Función clave                                | Qué hace                                                       |
| -------------------------------------------- | -------------------------------------------------------------- |
| `df.stat.crosstab(col1, col2)`               | Frecuencia entre dos columnas categóricas                      |
| `df.stat.corr(col1, col2, method="pearson")` | Correlación de Pearson o Spearman                              |
| `df.groupBy(...).count()`                    | También puede usarse para agrupaciones categóricas simples     |
| `display(df)`                                | Permite hacer scatter plot para ver relación entre 2 numéricas |


# Compare and contrast imputing missing values with the mean or median or mode value
(Impute missing values with the mode, mean, or median value)

Imputar = reemplazar valores faltantes (nulls / NaNs) con algún valor que represente el resto de la distribución.
¿Por qué imputar?

- Muchos modelos no aceptan datos nulos.
- Evita perder información si eliminas filas.
- El método elegido afecta el sesgo y la varianza del modelo.

| Método               | Se aplica a                                 | Ventajas                                             | Desventajas                                      | Cuándo usar                                                  |
| -------------------- | ------------------------------------------- | ---------------------------------------------------- | ------------------------------------------------ | ------------------------------------------------------------ |
| **Mean (media)**     | Variables numéricas continuas               | Fácil de calcular, mantiene media de la distribución | Afectado por *outliers* y sesgo                  | Cuando la variable tiene distribución **simétrica**          |
| **Median (mediana)** | Variables numéricas continuas               | Robusta a *outliers* y sesgo                         | No conserva la media original                    | Cuando la variable tiene **distribución sesgada o outliers** |
| **Mode (moda)**      | Variables categóricas o numéricas discretas | Mejor para categorías; representa el valor más común | No siempre es representativo si hay varias modas | Cuando la variable es **categórica o discreta**              |


## A. Con .fillna() → uso directo, manual

In [0]:
df.fillna({"age": 30, "gender": "Male"})

## B. Con Imputer (Spark ML) → solo numéricas

In [0]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=["age", "income"],
    outputCols=["age_imputed", "income_imputed"]
).setStrategy("mean")  # o "median"

model = imputer.fit(df)
df_imputed = model.transform(df)


## 📌 Puntos clave

| Concepto                                    | Detalle clave                                         |
| ------------------------------------------- | ----------------------------------------------------- |
| `mode` no está en `Imputer`                 | Debes calcularlo manualmente y usar `.fillna()`       |
| `median` es más robusta que `mean`          | Se usa cuando hay outliers o datos sesgados           |
| `mean` puede deformar la distribución       | Especialmente en variables de ingresos, precios, etc. |
| `Imputer` solo sirve con columnas numéricas | Para categóricas, debes usar otras técnicas           |
| Para variables categóricas                  | Solo **mode** es válido como imputación simple        |



# Use one-hot encoding for categorical features

(Identify and explain the model types or data sets for which one-hot encoding is or is not
appropriate.)

## Spark
En Spark, el one-hot encoding se hace típicamente en dos pasos con el módulo de MLlib:

In [0]:
# 1. Indexar la columna categórica (convertir a números)
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="color", outputCol="color_index")
df_indexed = indexer.fit(df).transform(df)

# 2. Aplicar OneHotEncoder
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCols=["color_index"], outputCols=["color_ohe"])
df_encoded = encoder.fit(df_indexed).transform(df_indexed)


## 📌 Puntos clave

| Punto                                                              | Explicación                                                              |
| ------------------------------------------------------------------ | ------------------------------------------------------------------------ |
| Puede crear muchas columnas si la variable tiene muchas categorías | → Aumenta dimensionalidad y puede causar overfitting                     |
| No es ideal para árboles de decisión o random forest en Spark      | → Spark ML maneja los índices de categoría directamente en estos modelos |
| No funciona con valores nulos                                      | → Debes imputar o filtrar primero                                        |


✅ Úsalo cuando:

La variable es categórica sin orden (nominal)
Usas modelos que no aceptan variables categóricas directamente, como regresión lineal, logística, redes neuronales

❌ Evita si:

Hay muchas categorías → usar feature hashing o embedding
Estás usando tree-based models en Spark ML (como Random Forest o GBT) → estos aceptan directamente índices de StringIndexer

# Identify scenarios where log scale transformation is appropriate


¿Qué es una transformación logarítmica?
Es una transformación matemática que aplica la función logaritmo (por ejemplo, log base 10 o log natural) a una variable:

In [0]:
from pyspark.sql.functions import log
df = df.withColumn("log_income", log(df["income"]))

Esta técnica reduce la escala de los valores grandes y puede ayudar a que los datos cumplan mejor los supuestos de ciertos modelos.

**Objetivo de la transformación logarítmica**
- Reducir sesgo hacia la derecha (right-skewed data)
- Estabilizar la varianza (heterocedasticidad)
- Mejorar relaciones lineales entre variables
- Permitir que el modelo aprenda mejor patrones exponenciales

**Casos típicos donde sí es apropiado aplicar log**
| Variable                                                  | Motivo                                                                     | Resultado esperado                         |
| --------------------------------------------------------- | -------------------------------------------------------------------------- | ------------------------------------------ |
| `income`, `house_price`, `sales`, `population`            | Datos muy **sesgados a la derecha**, con valores grandes y dispersión alta | Distribución más simétrica, más linealidad |
| `count` de eventos, como accesos por hora                 | Muchos ceros + valores extremos                                            | Compresión de valores grandes              |
| Relación exponencial (ej: y crece exponencialmente con x) | Mejora la linealidad                                                       | Permite usar regresión lineal              |

**Casos donde NO debes aplicar log**
| Caso                               | Por qué evitarlo                                |
| ---------------------------------- | ----------------------------------------------- |
| Valores ≤ 0 (negativos o ceros)    | No se puede aplicar log a 0 o números negativos |
| Variables ya normales o simétricas | No necesitas transformar                        |
| Variables categóricas              | No tiene sentido aplicar log                    |



## 📌 Puntos clave

| Situación                                                 | ¿Aplicar log? | Motivo                       |
| --------------------------------------------------------- | ------------- | ---------------------------- |
| Datos numéricos muy sesgados positivamente (right-skewed) | ✅ Sí          | Reduce la asimetría          |
| Rangos muy amplios                                        | ✅ Sí          | Normaliza la escala          |
| Exponenciales                                             | ✅ Sí          | Facilita modelado lineal     |
| Ceros o negativos                                         | ❌ No          | log(x) indefinido para x ≤ 0 |
| Categóricos                                               | ❌ No          | No tiene sentido             |
